In [1]:
from src.model import GPT,Config
from src.trainer import Trainer
import pandas as pd
import os
import torch
import numpy as np
import tiktoken
import multiprocessing as mp
import time

c:\Users\chris\.conda\envs\deep_learning_cuda\Lib\site-packages\torch\__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\Context.cpp:85.)
  _C._set_float32_matmul_precision(precision)


In [2]:
logpath = './log'
DATASET_PATH = './data/gutenberg'
SEED = 42

In [3]:

class DataLoaderLite:

    def __init__(self, B, T, process_rank, num_processes, split='train'):
        super().__init__()
        self.B, self.T = B, T
        self.process_rank = process_rank
        self.num_processes = num_processes
        assert split in {'train', 'val'}
        
        # get the shard filenames
        data_root = DATASET_PATH
        shard_filenames = os.listdir(data_root)
        shard_filenames = sorted([filename for filename in shard_filenames if split in filename])
        self.shard_filepaths = [os.path.join(data_root, filename) for filename in shard_filenames]
        assert len(self.shard_filepaths) > 0, f'no shards found for split {split}'
        master_process = process_rank == 0
        if master_process:
            print(f'found {len(self.shard_filepaths)} shards for split {split}')
        self.reset()

    def load_tokens(self, filepath):
        tokens = torch.tensor(np.load(filepath).astype(np.int32), dtype=torch.long)
        return tokens

    def reset(self):
        # state, init at shard 0
        self.curr_shard = 0
        self.tokens = self.load_tokens(self.shard_filepaths[self.curr_shard])
        self.curr_pos = self.B * self.T * self.process_rank

    def next_batch(self):
        B, T = self.B, self.T
        batch = self.tokens[self.curr_pos : self.curr_pos + B*T + 1]
        x_batch = batch[:-1].view(B, T)
        y_batch = batch[1:].view(B, T)
        self.curr_pos += B * T * self.num_processes
        if self.curr_pos + (B * T + 1) > len(self.tokens):
            self.curr_shard = (self.curr_shard + 1) % len(self.shard_filepaths)
            self.tokens = self.load_tokens(self.shard_filepaths[self.curr_shard])
            self.curr_pos = self.B * self.T * self.process_rank
        return x_batch, y_batch

In [4]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

device_type = 'cuda' if device.startswith('cuda') else 'cpu'
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

master_process = True

In [5]:
MINI_BATCH_SIZE = 6
CTX_LENGTH = 2048
NUM_HEADS = 8
NUM_LAYERS = 10
EMBED_DIM = 768
WEIGHT_DECAY =0.1
MAX_LR = 1e-3
MIN_LR = 1e-4
EVAL_FREQ = 1
MAX_STEPS = 2400
WARMUP_STEPS = 715

In [6]:
grad_accum_steps = 32

In [7]:
train_loader = DataLoaderLite(B=MINI_BATCH_SIZE, T=CTX_LENGTH, process_rank=0, num_processes=1, split='train')
val_loader = DataLoaderLite(B=MINI_BATCH_SIZE, T=CTX_LENGTH, process_rank=0, num_processes=1, split='val')

found 3 shards for split train
found 1 shards for split val


In [8]:
gpt_config = Config(vocab_size=50304,  # number of tokens: 50000 BPE merges + 256 bytes tokens + 1 <endoftext> token = 50257, 
                    # 50304 (nice number, lots of power of 2s) used instead of 50257 (bad, odd number)
                           context_length=CTX_LENGTH, 
                           num_layers=NUM_LAYERS, 
                           num_heads=NUM_HEADS, 
                           embedding_dim=EMBED_DIM
                           )

model = GPT(gpt_config)
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total number of trainable parameters: {total_params:,}')
model.to(device)
# model = torch.compile(model)
optimizer = model.configure_optimizer(weight_decay=WEIGHT_DECAY,lr=MAX_LR,device_type=device_type,master_process=master_process)
token_encoder = tiktoken.get_encoding('gpt2')


Total number of trainable parameters: 111,086,592
num decay parameter tensors: 42 with 110,985,216 parameters
num nodecay parameter tensors: 82 with 101,376 parameters
using fused AdamW optimizer: True


In [9]:
start_time = time.time()
trainer = Trainer(model, optimizer, train_loader, val_loader, token_encoder, EVAL_FREQ, grad_accum_steps, device,master_process, logpath)
history,evaluation = trainer.train(MAX_STEPS, WARMUP_STEPS, MAX_LR, MIN_LR)
dt = (time.time() - start_time) / (60*60)

step    0 | train loss: 10.98 | val loss: 10.90 | perplexity: 54435.09 | lr: 1.40e-06 | norm: 16.4292 | dt: 12362.8950ms | tok/sec: 31.8061
step    1 | train loss: 10.91 | val loss: 10.74 | perplexity: 46157.28 | lr: 2.80e-06 | norm: 16.4414 | dt: 15741.0264ms | tok/sec: 24.9803
step    2 | train loss: 10.75 | val loss: 10.53 | perplexity: 37282.40 | lr: 4.20e-06 | norm: 14.9576 | dt: 14234.7906ms | tok/sec: 27.6236
step    3 | train loss: 10.55 | val loss: 10.32 | perplexity: 30198.92 | lr: 5.59e-06 | norm: 12.9142 | dt: 14551.4734ms | tok/sec: 27.0224
step    4 | train loss: 10.35 | val loss: 10.13 | perplexity: 25173.52 | lr: 6.99e-06 | norm: 9.9737 | dt: 14153.6860ms | tok/sec: 27.7819
step    5 | train loss: 10.16 | val loss: 9.98 | perplexity: 21543.51 | lr: 8.39e-06 | norm: 8.8327 | dt: 14194.8779ms | tok/sec: 27.7013
step    6 | train loss: 9.98 | val loss: 9.85 | perplexity: 18890.57 | lr: 9.79e-06 | norm: 7.1139 | dt: 14202.7395ms | tok/sec: 27.6859
step    7 | train loss: 9.

In [10]:
print(f"Total training time: {dt:.4f}hr")

Total training time: 10.8067hr


In [11]:
import json

In [12]:
with open("eval_metrics.json", "w") as f:
    json.dump(evaluation, f, indent=4)

with open('training_history.json','w') as f:
    json.dump(history, f, indent=4)